In [15]:
import matplotlib.pyplot as plt
import cv2
import yaml

import torch

import warnings
warnings.simplefilter("ignore")

In [33]:
from src.utils import plot_image
from tqdm import tqdm

filename = "data/raw/20240602 212200 [FCtTboJZqRo].mp4"
# yolov5 = torch.hub.load("ultralytics/yolov5", "custom", "yolov5/weights/yolov5s.pt")
# name_dict = yaml.safe_load(open("yolov5/data/coco.yaml", "r"))["names"]

capture = cv2.VideoCapture(filename)
frame_num = 0
has_ball_frames = []
shown = 0

while capture.isOpened():
    ret, frame = capture.read()
    if not ret:
        break

    if frame_num < 2000:
        frame_num += 1
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = yolov5(frame_rgb)
    results = (
        results
        .xyxy[0]
        .detach()
        .cpu()
        .numpy()
        .astype(int)
    )
    for x1, y1, x2, y2, conf, name_id in results:
        name = name_dict[int(name_id)]
        if name != "sports ball":
            cv2.imshow("stream", frame)
            continue

        annotated_frame = frame.copy()
        cv2.rectangle(
            annotated_frame,
            (x1, y1),
            (x2, y2),
            color = (0, 0, 255),
            thickness = 1
        )
        cv2.imshow("stream", annotated_frame)

    cv2.setWindowProperty("stream", cv2.WND_PROP_TOPMOST, 1)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

    frame_num += 1

cv2.destroyAllWindows()

Yolov5 is not accurate enough to pickup the position of the ball. Will need to finetune with respect to the dataset we have.